Word Embedding chapter 2.1 assignment


In [ ]:
with open('corpus/the-verdict.txt', 'r', encoding='utf-8') as file:
    raw_text = file.read()
print("Total number of characters in the file:", len(raw_text))
print(raw_text[:100])









In [ ]:
import re 
text='hello, world! a;lkjf i know htat if happens'
result=re.split(r'(\s)',text)
print(result)









In [ ]:
import re 
text='hello, world! a;lkjf i know htat if happens'
result2=re.split(r'([\s,;])',text)
print(result2)









In [ ]:
import re

# Read the text from the file
with open('corpus/the-verdict.txt', 'r', encoding='utf-8') as f:
    verdict_text = f.read()

# Use regex to split so that every word, whitespace, and special character is a separate token
# This will match words, whitespace, or any single non-whitespace, non-word character
tokens = re.findall(r'\w+|\s+|[^\w\s]', verdict_text)

# Print the first 50 tokens as a check
print(tokens[:500])



In [ ]:
tokens_no_whitespace = [token for token in tokens if not token.isspace()]
print(tokens_no_whitespace[:500])


In [ ]:
len(tokens_no_whitespace)

In [ ]:
all_words=sorted(set(tokens_no_whitespace))
vocab_size=len(all_words)
print(f"Vocabulary size: {vocab_size}")

In [ ]:
word_to_index = {word: idx for idx, word in enumerate(all_words)}
print(list(word_to_index.items())[:50])


In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_idx = {word: idx for idx, word in enumerate(vocab)}
        self.idx_to_str = {idx: word for word, idx in self.str_to_idx.items()}
        self.unk_token = "<unk>" if "<unk>" in self.str_to_idx else None

    def encode(self, text):
        preprocessed_text = re.sub(r'[^\w\s]', '', text)
        words = preprocessed_text.split()
        if self.unk_token is not None:
            return [self.str_to_idx[word] if word in self.str_to_idx else self.str_to_idx[self.unk_token] for word in words]
        else:
            return [self.str_to_idx[word] for word in words]

    def decode(self, ids):
        text = ' '.join([self.idx_to_str[idx] if idx in self.idx_to_str else (self.unk_token if self.unk_token is not None else '') for idx in ids])
        text = re.sub(r'\s+', ' ', text)
        return text.strip()


In [ ]:
# Let's test the SimpleTokenizerV1 class with a sample text
# Ensure the sample text only contains words from the vocab
sample_text = "I had always thought Jack Gisburn rather a cheap good fellow enough so it was no great surprise to me to hear that in the height of his glory he had dropped his painting married a rich widow and established himself in a villa on the Riviera"
tokenizer = SimpleTokenizerV1(all_words)

# Encode the sample text
encoded = tokenizer.encode(sample_text)
print("Encoded:", encoded)

# Decode the encoded ids
decoded = tokenizer.decode(encoded)
print("Decoded:", decoded)


In [ ]:
# Add special tokens to the vocabulary and update the mappings
special_tokens = ['<eot>', '<unk>']
for token in special_tokens:
    if token not in all_words:
        all_words.append(token)

# Update vocab_size
vocab_size = len(all_words)
print(f"Vocabulary size (with special tokens): {vocab_size}")



# Show the last few entries to confirm special tokens are present
print("Last 5 vocab entries:", list(word_to_index.items())[-5:])


In [ ]:
text1='hello, world! a;lkjf i know htat if happens'
text2="In the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera."

text="<eot>".join([text1,text2])
print(text)











In [ ]:
tokenizer=SimpleTokenizerV1(all_words)

print(tokenizer.encode(text))










In [ ]:
import tiktoken

# Common choices:
# - "cl100k_base": used by GPT-4/3.5 Turbo
# - "gpt2": used by GPT‑2/3 legacy
enc = tiktoken.get_encoding("cl100k_base")

text = "Hello, world! The Verdict by Edith Wharton."
ids = enc.encode(text)                 # list of token ids
print(ids, len(ids))
print(enc.decode(ids))                 # back to string

In [ ]:
decoded_text = enc.decode(ids)
print(decoded_text)


In [ ]:
with open('corpus/the-verdict.txt', 'r') as file:
    raw_text = file.read()


enc_text=enc.encode(raw_text)
print(len(enc_text))




In [ ]:
sample_100 = enc_text[:100]
print(sample_100)


In [ ]:
context_size=4

x=enc_text[:context_size]


In [ ]:
import torch
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)  #1

        for i in range(0, len(token_ids) - max_length, stride):  #2
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):  #3
        return len(self.input_ids)

    def __getitem__(self, idx):  #4
        return self.input_ids[idx], self.target_ids[idx]
        

In [ ]:
dataset=GPTDatasetV1(txt=raw_text,tokenizer=enc,max_length=4,stride=10)

In [ ]:
len(dataset)




In [ ]:
import torch
import torch.nn as nn

# Define a simple embedding layer
vocab_size =50257 # assuming enc_text contains all token ids
embedding_dim = 256  # you can choose any dimension

embedding_layer = nn.Embedding(vocab_size, embedding_dim)

# Example: get embeddings for the first 10 tokens
input_tokens = torch.tensor(enc_text[:10])
embeddings = embedding_layer(input_tokens)
print(embeddings)


In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)


In [ ]:
token_embeddings=embedding_layer(inputs)
print(token_embeddings.shape)

absolute embeddings layer

In [ ]:
context_length=max_length
pos_embedding_layer=torch.nn.Embedding(context_length,embedding_dim)
positional_embeddings=pos_embedding_layer(torch.arange(context_length))
print(positional_embeddings)

In [ ]:
input_embeddings=positional_embeddings+token_embeddings

print(input_embeddings.shape)

In [ ]:
query = torch.empty(input_embeddings.shape)
query.shape
attention_table_col=input_embeddings.shape[1]
attention_table_row=input_embeddings.shape[1]
attention_scores = torch.empty(8,attention_table_row,attention_table_col)
attention_weights = torch.empty(8,attention_table_row,attention_table_col)


for i,element in enumerate(input_embeddings):
    attention_scores[i]=element@element.T
    attention_weights[i]=torch.softmax(attention_scores[i],dim=1)
    # print(attention_weights)
    print(attention_weights[i].sum(dim=1))
    print(attention_weights[i].shape)
    # print(attention_weights.shape)
    query[i]=attention_weights[i]@element
    




In [ ]:
torch.manual_seed(72)
d_in=256
d_out=72
W_query=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_key=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_value=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)




In [ ]:
input_vector = input_embeddings[0][0]
print(input_vector.reshape(1, -1))

In [ ]:
input_to_query=input_vector@W_query
input_to_key=input_vector@W_key
input_to_value=input_vector@W_value


In [ ]:
print(input_to_query.shape)

In [ ]:
inputs_to_querys=input_embeddings@W_query
inputs_to_keys=input_embeddings@W_key
inputs_to_values=input_embeddings@W_value

In [ ]:
print(inputs_to_querys.shape)
print(inputs_to_keys.shape)

In [ ]:
query_key_attention_scores=inputs_to_querys@inputs_to_keys.transpose(1,2)

In [ ]:

d_k=inputs_to_keys.shape[-1]
print(d_k)
query_key_attention_scores_normalized=torch.softmax(query_key_attention_scores/d_k**0.5,dim=-1)

In [ ]:
context_vector=query_key_attention_scores_normalized@inputs_to_values

In [ ]:
print(context_vector.shape)

In [ ]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self,d_in,d_out):
        super.__init__()
        self.W_query=torch.nn.Parameter(torch.rand(d_in,d_out))
        self.W_key=torch.nn.Parameter(torch.rand(d_in,d_out))
        self.W_value=torch.nn.Parameter(torch.rand(d_in,d_out))
        
        
    def forward(self,x):
        queries=x@self.W_query
        keys=x@self.W_key
        values=x@self.W_values
        
        attention_scores=queries@keys.T
        attention_weights=torch.softmax(attention_scores/keys.shape[-1]**0.5,dim=-1)
        context_vectors=attention_weights@values
        return context_vectors
        
        



In [ ]:

#  Just used nn.Linear for better activation. 



class SelfAttention_v2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):
        super.__init__()
        self.W_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value=nn.Linear(d_in,d_out,bias=qkv_bias)
    def forward(self,X):
        queries=x@self.W_query
        keys=x@self.W_key
        values=x@self.W_values
        
        attention_scores=queries@keys.T
        attention_weight=torch.softmax(attention_scores/keys.shape[-1]**0.5,dim=-1)
        context_vectors=attention_weight@values
        return context_vectors
        

In [ ]:
class CausalAttention(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):
        super.__init__()
        self.W_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value=nn.Linear(d_in,d_out,bias=qkv_bias)
    
    def forward(self,x):
        queries=x@self.W_query
        keys=x@self.W_key
        values=x@self.W_value
        
        attention_scores=queries@keys.T
        mask=torch.ones(attention_scores.shape[0],attention_scores.shape[1])
        mask=torch.triu(mask)
        attention_scores[mask==1]=torch.tensor(-float('inf'))
        attention_weight=torch.softmax(attention_scores/keys.shape[-1]**0.5,dim=-1)
        # drop out here
        # The 'train' parameter specifies whether dropout should behave in training mode (drop values) or evaluation mode (no dropout).
        # In PyTorch, when train=True, dropout is applied; when train=False, dropout is bypassed.
        attention_weight = torch.dropout(attention_weight, p=0.5, train=self.training)
        context_vectors=attention_weight@values
        return context_vectors

In [ ]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self,d_in,d_out,context_length,drop_out,num_heads,qkv_bias=False):
        super().__init__()
        self.heads=nn.ModuleList([CausalAttention(d_in,d_out,context_length,drop_out,qkv_bias)for _ in range(num_heads)])
    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim=-1)
        

In [ ]:
# The code in the image and your code are very similar, but there are a few key differences and some mistakes in your implementation.
# Here is a version that matches the code in the image, with comments on the differences:

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, drop_out, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        self.d_out = d_out  
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.drop_out = nn.Dropout(drop_out)
        self.out_proj = nn.Linear(d_out, d_out)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # Linear projections
        keys = self.W_key(x)      # (b, num_tokens, d_out)
        values = self.W_value(x)  # (b, num_tokens, d_out)
        queries = self.W_query(x) # (b, num_tokens, d_out)

        # Reshape for multi-head: (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose to (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Attention score: (b, num_heads, num_tokens, num_tokens)
        attn_scores = queries @ keys.transpose(2, 3)

        # Apply mask (causal)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores = attn_scores.masked_fill(mask_bool, -torch.inf)

        # Scale and softmax
        attn_weights = torch.softmax(attn_scores / (self.head_dim ** 0.5), dim=-1)
        attn_weights = self.drop_out(attn_weights)

        # Weighted sum
        context_vec = (attn_weights @ values).transpose(1, 2)  # (b, num_tokens, num_heads, head_dim)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec


In [ ]:
gpt_configuration = {
    "vocab_size": 50257,
    "context_length": 256,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

In [ ]:
import torch 
import torch.nn as nn

class Dummy_Scratch_gpt_model(nn.Module):
    def __init__(self,configuration):
        
        super().__init__()
        
        self.token_embedding=nn.Embedding(configuration["vocab_size"],configuration["emb_dim"])
        self.pos_emb=nn.Embedding(configuration["context_length"],configuration["emb_dim"])
        self.drop_emb=nn.Dropout(configuration["drop_rate"])
        
        self.transformer_blocks=nn.Sequential(
            *[Dummy_Transformer_Block(configuration) for _ in range(configuration["n_layers"])]
        )
    
    
        self.final_norm=Dummay_Layer_normalization(configuration["emb_dim"])
        self.out_head=nn.Linear(configuration["emb_dim"],configuration["vocab_size"],bias=False)
        
        
    def forward(self,in_dx):
        batch_size,seq_len=in_dx.shape
        tok_embeddings=self.token_embedding(in_dx)
        pos_embeddings=self.pos_emb(torch.arange(seq_len,device=in_dx.device))
        x=tok_embeddings+pos_embeddings
        x=self.drop_emb(x)
        x=self.transformer_blocks(x)
        x=self.final_norm(x)
        logits=self.out_head(x)
        return logits
        

class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi, device=x.device)) * (x + 0.044715 * torch.pow(x, 3))))

class Dummy_FeedForward(nn.Module):
    def __init__(self, emb_dim, hidden_dim=None, drop_rate=0.1):
        super().__init__()
        if hidden_dim is None:
            hidden_dim = emb_dim * 4
        self.fc1 = nn.Linear(emb_dim, hidden_dim)
        self.gelu = GELU()
        self.fc2 = nn.Linear(hidden_dim, emb_dim)
        self.dropout = nn.Dropout(drop_rate)
    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

class Dummy_SelfAttention(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.mha = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            drop_out=cfg["drop_rate"],
            num_heads=cfg["n_heads"],
            qkv_bias=cfg["qkv_bias"]
        )
    def forward(self, x):
        return self.mha(x)

class Dummy_Transformer_Block(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.ln1 = Dummay_Layer_normalization(cfg["emb_dim"])
        self.attn = Dummy_SelfAttention(cfg)
        self.ln2 = Dummay_Layer_normalization(cfg["emb_dim"])
        self.ff = Dummy_FeedForward(cfg["emb_dim"], drop_rate=cfg["drop_rate"])
    def forward(self, x):
        # Attention block with residual
        x = x + self.attn(self.ln1(x))
        # Feedforward block with residual
        x = x + self.ff(self.ln2(x))
        return x

class Dummay_Layer_normalization(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps=eps
        self.scale=nn.Parameter(torch.ones(emb_dim))
        self.shift=nn.Parameter(torch.zeros(emb_dim))
    def forward(self, x):
        mean=x.mean(-1,keepdim=True)
        var=x.var(-1,keepdim=True, unbiased=False)
        norm_x=(x-mean)/torch.sqrt(var+self.eps)
        return self.scale*norm_x+self.shift
    

In [ ]:
import tiktoken

tokenizer=tiktoken.get_encoding("gpt2")

batch=[]
str1 = "This is the first string."
str2 = "This is the second string."

batch.append(tokenizer.encode(str1))
batch.append(tokenizer.encode(str2))

In [ ]:
torch.manual_seed(22)
model = Dummy_Scratch_gpt_model(gpt_configuration)
total_params = sum(p.numel() for p in model.parameters())
print("Total parameter count:", total_params)

# Count parameters in all transformer blocks
transformer_params = 0
ff_params = 0

for name, module in model.named_modules():
    if isinstance(module, Dummy_Transformer_Block):
        # All params in this transformer block
        transformer_params += sum(p.numel() for p in module.parameters())
        # Params in the feedforward submodule of this block
        ff_params += sum(p.numel() for p in module.ff.parameters())

print("Total parameters in all transformer blocks:", transformer_params)
print("Total parameters in all feedforward networks inside transformer blocks:", ff_params)


In [ ]:
# Compute the memory usage of the model (in megabytes)
memory_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
memory_mb = memory_bytes / (1024 * 1024)
print("Total model memory (MB):", memory_mb)


In [ ]:

sample_input = torch.randn(2, 5, 768)  # (batch, seq_len, emb_dim)
transformer_block = Dummy_Transformer_Block(gpt_configuration)
output = transformer_block(sample_input)
print("Input shape:", sample_input.shape)
print("Output shape:", output.shape)


In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

gelu = GELU()

x = torch.linspace(-5, 5, 1000)
y = gelu(x)

plt.figure(figsize=(8, 5))
plt.plot(x.numpy(), y.detach().numpy(), label="GELU")
plt.title("GELU Activation Function")
plt.xlabel("x")
plt.ylabel("GELU(x)")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
def inference_time_prediction(model,input_sequence,max_token_window,context_size):
    
    for _ in range(max_token_window):
        input_sequence_to_model=input_sequence[:,-context_size:]
        with torch.no_grad():
            logits=model(input_sequence)
        logits=logits[:,-1,:]
        # probs=logits
        logits=torch.argmax(torch.softmax(logits,dim=-1),dim=-1,keepdim=True)
        input_sequence=torch.cat((input_sequence,logits),dim=1)
        
    return input_sequence
        
            

In [ ]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # #1 Adds batch dimension
print("encoded_tensor.shape:", encoded_tensor.shape)


In [ ]:
model.eval()

out=inference_time_prediction(model,encoded_tensor,15,gpt_configuration["context_length"])

print(out)
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)


In [ ]:
import torch


GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256,  # 1
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,       # 2
    "qkv_bias": False
}

torch.manual_seed(123)
model = Dummy_Scratch_gpt_model(GPT_CONFIG_124M)
model.eval()


In [ ]:
import tiktoken

def text_to_token(text,tokenizer):
    encoded=tokenizer.encode(text,allowed_special={'<|endoftext|>'})
    encoded_tensor=torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor
def token_to_text(token,tokenizer):
    flat=token.squeeze(0)
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = inference_time_prediction(
        model,
        text_to_token(start_context, tokenizer),
        10,
        GPT_CONFIG_124M["context_length"]
    )
print("Output text:\n", token_to_text(token_ids, tokenizer))

        

In [ ]:
# Generate some text and call the model

# Define a new prompt for generation
prompt = "The future of artificial intelligence is"

# Tokenize the prompt and convert to tensor
tokenizer = tiktoken.get_encoding("gpt2")
encoded = tokenizer.encode(prompt)
prompt_tensor = torch.tensor(encoded[:5]).unsqueeze(0)  # Add batch dimension

# Now pass the tensor to the model
probs = model(prompt_tensor)
# Implement similar logic to get the probabilities of the target tokens in the generated text

import torch.nn.functional as F

# Get the model's output probabilities (logits -> probabilities)
probs_softmax = F.softmax(probs, dim=-1)

# For demonstration, let's get the probability of the next token at each position in the prompt
# The "target" for each position is the next token in the prompt (shifted by 1)
# For the last token, there is no next token, so we limit to len(encoded) - 1
targets = encoded[1:]  # next tokens
token_indices = list(range(len(targets)))  # positions 0 to len(targets)-1
batch_idx = 0  # batch size is 1

# Gather the probabilities for the target tokens at each position
target_probas = probs_softmax[batch_idx, token_indices, targets]
print("Target token probabilities for each position:", target_probas)

In [ ]:
import torch

# Compute the log and then mean of the target probabilities
log_target_probas = torch.log(target_probas)
mean_log_target_probas = -1*log_target_probas.mean()
print("Mean log target probability:", mean_log_target_probas)


In [ ]:
print("probs shape:", probs.shape)
print("targets shape:", torch.tensor(targets).shape)
loss = torch.nn.functional.cross_entropy(probs.flatten(0,1), torch.tensor(targets))

In [ ]:
print(loss)

In [ ]:
# Tokenize the raw_text and print the vocabulary size
tokenizer = tiktoken.get_encoding("gpt2")
tokens = tokenizer.encode(raw_text)
print("Number of tokens in raw_text:", len(tokens))
print("Vocabulary size:", tokenizer.n_vocab)

In [ ]:
training_ratio = 0.9
split_index = int(training_ratio * len(raw_text))
train_data = raw_text[:split_index]
test_data = raw_text[split_index:]

# Create dataloaders for train and test
train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=gpt_configuration["context_length"],
    stride=gpt_configuration["context_length"],
    shuffle=True,
    num_workers=0
)
test_loader = create_dataloader_v1(
    test_data,
    batch_size=2,
    max_length=gpt_configuration["context_length"],
    stride=gpt_configuration["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [ ]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)
    
    

print("\nValidation loader:")
for x, y in test_loader:
    print(x.shape, y.shape)
    


In [ ]:
def calc_loss_batch(input_batch,output_batch,device,model):
    input_batch=input_batch.to(device)
    output_batch=output_batch.to(device)
    probs=model(input_batch)
    loss=torch.nn.functional.cross_entropy(probs.flatten(0,1),output_batch.flatten())
    return loss

In [ ]:
def calc_loss_on_loader(loader, device, model):
    model.eval()
    total_loss = 0.0
    total_batches = 0
    with torch.no_grad():
        for input_batch, output_batch in loader:
            loss = calc_loss_batch(input_batch, output_batch, device, model)
            total_loss += loss.item()
            total_batches += 1
    avg_loss = total_loss / total_batches if total_batches > 0 else 0.0
    return avg_loss


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
with torch.no_grad():
    train_loss = calc_loss_on_loader(train_loader, device, model)
    val_loss = calc_loss_on_loader(test_loader, device, model)
print("Training loss:", train_loss)
print("Validation loss:", val_loss)


In [ ]:
def evaluate_model(model,train_loader,test_loader,device,eval_iter):
    model.to(device)
    model.eval()
    
    with torch.no_grad():
        train_loss = calc_loss_on_loader(train_loader, device, model)
        val_loss = calc_loss_on_loader(test_loader, device, model)
        
    model.train()
    return train_loss,val_loss

In [ ]:
def generate_and_print_sample(model,tokenizer,device,start_context):
    model.eval()
    context_size=model.pos_emb.weight.shape[0]
    encoded=text_to_token(start_context,tokenizer).to(device)
    with torch.no_grad():
        token_ids=inference_time_prediction(model=model,input_sequence=encoded,max_token_window=50,context_size=context_size)
    decoded_text=token_to_text(token_ids,tokenizer)
    print(decoded_text)
    model.train()
    
    

In [ ]:
def train_model(model,optimizer,tokenizer,device,no_of_epoches,train_loader,test_loader,eval_iter,eval_frequency):
    
    trainlosses,testlosses,track_token_seen=[],[],[]
    token_seen,global_step=0,-1
    
    for epoches in range(no_of_epoches):
        model.train()
        for input_batch,target_batch in train_loader:
            optimizer.zero_grad()
            loss=calc_loss_batch(input_batch,target_batch,device,model)
            loss.backward()
            optimizer.step()
            token_seen+=input_batch.numel()
            global_step+=1
            
            if global_step % eval_frequency==0:
            
                train_loss, test_loss=evaluate_model(model,train_loader,test_loader,device,eval_iter)
                trainlosses.append(train_loss)
                testlosses.append(test_loss)
                
                track_token_seen.append(token_seen)
                print(f"Ep {epoches+1}: (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, "
                      f"val loss {test_loss:.3f}"
                )
                generate_and_print_sample(
                    model, tokenizer, device, start_context
                )
                
    return trainlosses,testlosses,track_token_seen
        

In [ ]:
torch.manual_seed(123)
model = Dummy_Scratch_gpt_model(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=0.0004, weight_decay=0.1
)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model(
    model, optimizer, tokenizer, device,
    no_of_epoches=num_epochs,
    train_loader=train_loader,
    test_loader=test_loader,
    eval_iter=5,
    eval_frequency=5
)
